In [284]:
import sympy as smp
import numpy as np
from scipy.optimize import  curve_fit

import pandas as pd
import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'notebook', 'grid'])

In [285]:
#massas e comprimentos dos carrinhos

#massas
m1 = 196.02 *1e-03
m2 = 196.02 *1e-03
delta_m1 = 0.52 *1e-03

m_err = 0.01 *1e-03

#comprimentos
L_1 = 120.00 *1e-03
L_2 = 120.15 *1e-03
L_err = 0.05 *1e-03

#erros das medições de tempo

delta_t_err = 0.000001

In [286]:
def velocidades(delta_t_2, delta_t_1):
    #só carrinho dois se move inicialmente

    v_2 = L_2/delta_t_2
    v_2_err = (1/delta_t_2) *L_err + (L_2/(delta_t_2**2)) *delta_t_err

    #só carrinho um se move no fim

    v_1 = L_1/delta_t_1
    v_1_err = (1/delta_t_1) *L_err + (L_1/(delta_t_1**2)) *delta_t_err

    #resultados
    print("A velocidade inicial do carrinho 2 é: " + str(v_2) + ' +- ' + str(v_2_err))
    print("A velocidade final do carrinho 1 é: " + str(v_1) + ' +- ' + str(v_1_err))

    return v_2, v_2_err, v_1, v_1_err

def velocidade_cm(v_2, v_2_err, v_1, v_1_err, elastico = True):
    #velocidade do cm

    v_cm_i = m2*v_2/(m1 + m2)
    v_cm_i_err = m_err*v_2/(m1 + m2) + m2*v_2_err/(m1 + m2) + m2*v_2/((m1 + m2)**2) *2*m_err

    if elastico:
        v_cm_f = v_1/(m1 + m2)
        v_cm_f_err = m_err*v_1/(m1 + m2) + m1*v_1_err/(m1 + m2) + m1*v_1/((m1 + m2)**2) *2*m_err
    else:
        v_cm_f  = v_1
        v_cm_f_err = v_1_err

    print('')
    print('A velocidade do CM antes da colisão é: ' + str(v_cm_i) + ' +- ' + str(v_cm_i_err))
    print('A velocidade do CM depois da colisão é: ' + str(v_cm_f) + ' +- ' + str(v_cm_f_err))
    print('')

    return v_cm_i, v_cm_i_err

def velocidade_referencial_CM(v_2, v_2_err, v_1, v_1_err, elastico):
    v_cm_i, v_cm_i_err = velocidade_cm(v_2, v_2_err, v_1, v_1_err, elastico)

    v_2_i_cm = v_2 - v_cm_i
    v_2_i_cm_err = v_2_err + v_cm_i_err

    v_1_i_cm = -v_cm_i
    v_1_i_cm_err = v_cm_i_err

    v_1_f_cm = v_1 - v_cm_i
    v_1_f_cm_err = v_1_err + v_cm_i_err

    if elastico:
        v_2_f_cm = -v_cm_i
        v_2_f_cm_err = v_cm_i_err
    else:
        v_2_f_cm = v_1_f_cm
        v_2_f_cm_err = v_1_f_cm_err

    #resultados
    print("A velocidade inicial do carrinho 2 no CM é: " + str(v_2_i_cm) + ' +- ' + str(v_2_i_cm_err))
    print("A velocidade inicial do carrinho 1 no CM é: " + str(v_1_i_cm) + ' +- ' + str(v_1_i_cm_err))

    print("A velocidade final do carrinho 2 no CM é: " + str(v_2_f_cm) + ' +- ' + str(v_2_f_cm_err))
    print("A velocidade final do carrinho 1 no CM é: " + str(v_1_f_cm) + ' +- ' + str(v_1_f_cm_err))
    
    return v_2_i_cm, v_2_i_cm_err, v_1_i_cm, v_1_i_cm_err, v_2_f_cm, v_2_f_cm_err, v_1_f_cm, v_1_f_cm_err

In [287]:
def analise(v_2_i, v_2_i_err, v_1_i, v_1_i_err, v_2_f, v_2_f_err, v_1_f, v_1_f_err):

    #cálculo dos momentos

    p_i= m2*v_2_i + m1*v_1_i
    p_i_err = m_err*np.abs(v_2_i) + m2*v_2_i_err + m_err*np.abs(v_1_i) + m1*v_1_i_err

    p_f= m2*v_2_f + m1*v_1_f
    p_f_err = m_err*np.abs(v_2_f) + m2*v_2_f_err + m_err*np.abs(v_1_f) + m1*v_1_f_err

    delta_p = p_f - p_i
    delta_p_err = p_f_err + p_i_err

    try:
        variacao_percentual = 100 * delta_p/p_i
        variacao_percentual_err = 100*np.abs(delta_p_err)/np.abs(p_i) + 100*(np.abs(delta_p)/(p_i**2)) * p_i_err
    except:
        variacao_percentual =  'Não existe' 
        variacao_percentual_err = 'Momento incial = 0'

    #Impulso

    I_2 = m2*(v_2_f -v_2_i)
    I_2_err = m_err*np.abs(v_2_f -v_2_i) + m2*(v_2_f_err + v_2_i_err)

    I_1 = m1*(v_1_f -v_1_i)
    I_1_err = m_err*np.abs(v_1_f -v_1_i) + m2*(v_1_f_err + v_1_i_err)

    print('')
    print('A variação percentual dos momentos é: ' + str(variacao_percentual) + ' +- ' + str(variacao_percentual_err))
    print('O impulso total do sistema é: ' + str(delta_p) + ' +- ' + str(delta_p_err))
    print('O impulso no carrinho 2 é: ' + str(I_2) + ' +- ' + str(I_2_err))
    print('O impulso no carrinho 1 é: ' + str(I_1) + ' +- ' + str(I_1_err))

    #força média

    F_2 = I_2/(1e-3)
    F_2_err = I_2_err/(1e-3)

    F_1 = I_1/(1e-3)
    F_1_err = I_1_err/(1e-3)

    print('')
    print('A força média no carrinho 2 é: ' + str(F_2) + ' +- ' + str(F_2_err))
    print('A força média no carrinho 1 é: ' + str(F_1) + ' +- ' + str(F_1_err))


    #coeficiente de restituição

    restituicao = - (v_2_f - v_1_f)/(v_2_i - v_1_i)
    restituicao_err = (v_2_f_err + v_1_f_err)/np.abs((v_2_i - v_1_i)) +  np.abs(v_2_f - v_1_f)/((v_2_i - v_1_i)**2) *(v_2_i_err + v_1_i_err)

    print('')
    print('O coeficiente de restituição é: ' + str(restituicao) + ' +- ' + str(restituicao_err))

    #energia cinética

    E_i = (1/2) *m2 * v_2_i**2 + (1/2) *m1 * v_1_i**2
    E_i_err = (1/2) *m_err * v_2_i**2 + m2*np.abs(v_2_i)*v_2_i_err + (1/2) *m_err * v_1_i**2 + m1*np.abs(v_1_i)*v_1_i_err

    E_f = (1/2) *m2 * v_2_f**2 + (1/2) *m1 * v_1_f**2
    E_f_err = (1/2) *m_err * v_2_f**2 + m2*np.abs(v_2_f)*v_2_f_err + (1/2) *m_err * v_1_f**2 + m1*np.abs(v_1_f)*v_1_f_err

    delta_E = E_f - E_i
    delta_E_err = E_f_err + E_i_err
    variacao_percentual_energia = 100*delta_E/E_i
    variacao_percentual_energia_err = 100*delta_E_err/E_i + 100*(delta_E/(E_i**2))*E_i_err

    print('')
    print('A energia incial é: ' + str(E_i) + ' +- ' + str(E_i_err))
    print('A energia final é: ' + str(E_f) + ' +- ' + str(E_f_err))
    print('A variação percentual das energias é: ' + str(variacao_percentual_energia) + ' +- ' + str(variacao_percentual_energia_err))


In [288]:
def main(delta_t_2, delta_t_1, elastico = True):
    v_2_i, v_2_i_err, v_1_f, v_1_f_err = velocidades(delta_t_2, delta_t_1)
    v_2_i_cm, v_2_i_cm_err, v_1_i_cm, v_1_i_cm_err, v_2_f_cm, v_2_f_cm_err, v_1_f_cm, v_1_f_cm_err = velocidade_referencial_CM(v_2_i, v_2_i_err, v_1_f, v_1_f_err, elastico)
    print('')

    if elastico:
        #análise no referencial do laboratório
        print('ANÁLISE NO REFERENCIAL DO LABORATÓRIO')
        analise(v_2_i, v_2_i_err, 0, 0, 0, 0, v_1_f, v_1_f_err)
        print('')

        #análise no referencial do CM
        print('ANÁLISE NO REFERENCIAL DO CM')
        analise( v_2_i_cm, v_2_i_cm_err, v_1_i_cm, v_1_i_cm_err, v_2_f_cm, v_2_f_cm_err, v_1_f_cm, v_1_f_cm_err)
        print('')

    else:
        #análise no referencial do laboratório
        print('ANÁLISE NO REFERENCIAL DO LABORATÓRIO')
        analise(v_2_i, v_2_i_err, 0, 0, v_1_f, v_1_f_err, v_1_f, v_1_f_err)
        print('')

        #análise no referencial do CM
        print('ANÁLISE NO REFERENCIAL DO CM')
        analise( v_2_i_cm, v_2_i_cm_err, v_1_i_cm, v_1_i_cm_err, v_2_f_cm, v_2_f_cm_err, v_1_f_cm, v_1_f_cm_err)
        print('')
        

In [289]:
#choque elástico

delta_t_2_elastico = 0.112064
delta_t_1_elastico = 0.113774

main(delta_t_2_elastico, delta_t_1_elastico)

A velocidade inicial do carrinho 2 é: 1.0721551970302685 +- 0.0004557409622807527
A velocidade final do carrinho 1 é: 1.0547225200836745 +- 0.00044873804665462825

A velocidade do CM antes da colisão é: 0.5360775985151343 +- 0.0002825666956608471
A velocidade do CM depois da colisão é: 2.690344148769703 +- 0.0002781759063027082

A velocidade inicial do carrinho 2 no CM é: 0.5360775985151343 +- 0.0007383076579415998
A velocidade inicial do carrinho 1 no CM é: -0.5360775985151343 +- 0.0002825666956608471
A velocidade final do carrinho 2 no CM é: -0.5360775985151343 +- 0.0002825666956608471
A velocidade final do carrinho 1 no CM é: 0.5186449215685403 +- 0.0007313047423154754

ANÁLISE NO REFERENCIAL DO LABORATÓRIO

A variação percentual dos momentos é: -1.6259471571727966 +- 0.09525501047100422
O impulso total do sistema é: -0.0034171533350713656 +- 0.00019856475250265283
O impulso no carrinho 2 é: -0.21016386172187326 +- 0.00010005589539657584
O impulso no carrinho 1 é: 0.2067467083868019

In [290]:
#choque inelástico

delta_t_2_inelastico = 0.100774
delta_t_1_inelastico = 0.205576
m1 += 0.52*1e-3
main(delta_t_2_inelastico, delta_t_1_inelastico, elastico=False)

A velocidade inicial do carrinho 2 é: 1.1922718161430528 +- 0.000507990868836635
A velocidade final do carrinho 1 é: 0.5837257267385297 +- 0.00024605851717485763

A velocidade do CM antes da colisão é: 0.5953462436324669 +- 0.0003143621692058209
A velocidade do CM depois da colisão é: 0.5837257267385297 +- 0.00024605851717485763

A velocidade inicial do carrinho 2 no CM é: 0.5969255725105859 +- 0.0008223530380424559
A velocidade inicial do carrinho 1 no CM é: -0.5953462436324669 +- 0.0003143621692058209
A velocidade final do carrinho 2 no CM é: -0.011620516893937172 +- 0.0005604206863806785
A velocidade final do carrinho 1 no CM é: -0.011620516893937172 +- 0.0005604206863806785

ANÁLISE NO REFERENCIAL DO LABORATÓRIO

A variação percentual dos momentos é: -1.95189220024894 +- 0.09496534623260261
O impulso total do sistema é: -0.004561750111883978 +- 0.00021976633430772043
O impulso no carrinho 2 é: -0.11928720444507462 +- 0.00015389422154001803
O impulso no carrinho 1 é: 0.1147254543331